# Modeling Source-to-Sink systems using FastScape: 12. Sediment flux variations

![Namibian margin](Margin.jpg "Namibian margin")

In [ ]:
import xsimlab as xs
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('dark_background')
%load_ext xsimlab.ipython
import hvplot.xarray

## The marine model

FastScape contains a marine model developed by [Yuan et al, 2019](https://www.sciencedirect.com/science/article/pii/S0012821X19304200).

It assumes that sediment transport in the marine domain obeys a linear diffusion law.

The model is enriched by several factors/processes:
1. two different types of sediment are considered (sand and sill) that have different transport coefficient (diffusivity), typically 10-10$^3^$ m$^2$/yr for silt and 10$^0$-10$^3$ m$^2$/yr fro sand; one also needs to specify the sand-shale ratio of the source (out of the mountain)
2. the sediment is allowed to compact according to a compaction low of the form:

$$\phi=\phi_0e^{-z/z_0}$$

Typical value for $\phi_0$ are 0.63 for silt and 0.49 for sand and 2000 m and 3500 m for $z_0$ for silt and sand, respectively.

Note that here we will assume that silt and sand have the same transport coefficient which will ensure rapid convergence of the solution.

In [ ]:
from fastscape.models import marine_model

from fastscape.processes import (BlockUplift)

sink_model = (marine_model.
              drop_processes('diffusion').
              drop_processes('init_topography').
              drop_processes('uplift').
              update_processes({'uplift': BlockUplift}))

#sink_model.visualize(show_variables=True)


In [ ]:
xl = 100e3
yl = 100e3
nx = 101
ny = 101
X = np.linspace(0,xl,nx)
Y = np.linspace(0,yl,ny)
x,y = np.meshgrid(X, Y)

u0 = 3e-4
u = np.zeros((ny,nx))
ylim = 2*yl/(nx-1)
u = np.where(y<ylim, u0, 0)

h0 = np.where(y>ylim, -1000*(y-ylim)/(yl - ylim),0)
h0 = np.where(h0<-400, -400, h0)

In [ ]:
nstep = 201 # total number of steps
neq = 101 # number of steps to reach steady-state

teq = 1e7 # time to reach steady-state
period = 1e6 # period of climatic forcing
tfinal = teq + 5*period # final time

# Here we build the time array (note that not all time steps are of the same length)
tim1 = np.linspace(0,teq,101) 
tim2 = np.linspace(teq + period/10, tfinal, 100)
tim = np.concatenate((tim1,tim2))

# build precipitation array
uplift = 1 + 0.5*np.sin(2*np.pi*(tim-teq)/period)

u_tim_space = np.broadcast_to(u,(len(tim),ny,nx)).copy().transpose()
u_tim_space = np.where(u_tim_space>0,u_tim_space*uplift,u_tim_space).transpose()
u_xr = xr.DataArray(data=u_tim_space, dims=['time','y', 'x'])

fig, ax = plt.subplots(nrows = 1, ncols = 1, sharex=False, sharey=True, figsize=(12,7))

ax.plot(tim, uplift)



In [ ]:
# %create_setup sink_model --default --verbose
import xsimlab as xs

ds_in = xs.create_setup(
    model=sink_model,
    clocks={'time': tim,
           'strati': tim[::10]},
    master_clock='time',
    input_vars={
        # nb. of grid nodes in (y, x)
        'grid__shape': [ny,nx],
        # total grid length in (y, x)
        'grid__length': [yl,xl],
        # node status at borders
        'boundary__status': ['looped','looped','fixed_value','core'],
        # MFD partioner slope exponent
        'flow__slope_exp': 1,
        # drainage area exponent
        'spl__area_exp': 0.4,
        # slope exponent
        'spl__slope_exp': 1,
        # bedrock channel incision coefficient
        'spl__k_coef_bedrock': 1e-5,
        # soil (sediment) channel incision coefficient
        'spl__k_coef_soil': 1e-4,
        # detached bedrock transport/deposition coefficient
        'spl__g_coef_bedrock': 1,
        # soil (sediment) transport/deposition coefficient
        'spl__g_coef_soil': 1,
        # sand/silt ratio of continental sediment source
        'marine__ss_ratio_land': 0.5,
        # surface (reference) porosity of sand
        'marine__porosity_sand': 0.49,
        # surface (reference) porosity of silt
        'marine__porosity_silt': 0.63,
        # e-folding depth of exp. porosity curve for sand
        'marine__e_depth_sand': 3.5e3,
        # e-folding depth of exp. porosity curve for silt
        'marine__e_depth_silt': 2e3,
        # diffusivity (transport coefficient) for sand
        'marine__diffusivity_sand': 1e1,
        # diffusivity (transport coefficient) for silt
        'marine__diffusivity_silt': 1e1,
        # mean depth (thickness) of marine active layer
        'marine__layer_depth': 100,
        # surface topography elevation
        'topography__elevation': h0,
        # sea level (elevation)
        'sea__level': 0.0,
        # horizon freezing (deactivation) time
        'strati__freeze_time': tim,
        # uplift rate
        'uplift__rate': u_xr,
    },
    output_vars={'topography__elevation': 'time',
                'drainage__area': 'time',
                'strati__elevation': 'strati'}
)


In [ ]:
with xs.monitoring.ProgressBar():
    ds_out = ds_in.xsimlab.run(model=sink_model)

In [ ]:
from ipyfastscape import TopoViz3d

app = TopoViz3d(ds_out, canvas_height=600, time_dim="time")

app.components['background_color'].set_color('black')
app.components['vertical_exaggeration'].set_factor(5)
app.components['timestepper'].go_to_time(ds_out.time[-1])

app.show()

In [ ]:
fig, ax = plt.subplots(figsize=(12,8))

nout = 101
for iout in range(nout-1, -1, -1):
#    ds_out.strati__elevation.isel(strati=-1).isel(horizon=iout).sel(x=xl/2)[ds_out.y>ylim].plot()
    ds_out.strati__elevation.isel(strati=-1).isel(horizon=iout).mean('x')[ds_out.y>ylim].plot()

In [ ]:
fig, ax = plt.subplots(figsize=(12,8))

nout = 101
for iout in range(nout-1, -1, -1):
    ds_out.strati__elevation.isel(strati=-1).isel(horizon=iout).sel(y=yl/4).plot()